# Basic Q&A System

### Sentence Retrival

In [49]:
import math,numpy,json,re,nltk
import time,re,os.path,sys
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from collections import defaultdict
from numpy import multiply
from math import sqrt
from nltk.tokenize import word_tokenize
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize
import os.path as path
from collections import OrderedDict
from nltk.stem.wordnet import WordNetLemmatizer as WNL
from sklearn.feature_extraction import DictVectorizer
from nltk import FreqDist, DictionaryProbDist
from operator import add
import csv, ast

#### Load data from json file

In [50]:
t0 = time.time()
filename_ls = ['QA_test.json']
dataset = []
train_path = path.abspath('data/QA_test.json')

dataset = []
with open(train_path) as f:
    for line in f:
        dataset+=(json.loads(line))
print "Import Successful "
print "There are totally", len(dataset),'documents in this dataset'

Import Successful 
There are totally 42 documents in this dataset


#### Build modle and evaluate the result

In [72]:
stopwords = set(nltk.corpus.stopwords.words('english')) # wrap in a set() (see below)
stemmer = nltk.stem.PorterStemmer() 

def my_tokenizer(doc):
    terms = set()
    for token in nltk.word_tokenize(doc):
        if token not in stopwords: # 'in' and 'not in' operations are much faster over sets that lists
            terms.add(stemmer.stem(token.lower()))
    return list(terms)


class MostRelevantSentenceModel(object):
    def __init__(self, vectorizer, collection_matrix):
        self.vectorizer = vectorizer
        self.collection_matrix = collection_matrix
        feature_array = vectorizer.get_feature_names()
        self.features = dict()
        for index in range(len(feature_array)):
            term = feature_array[index]
            self.features[term] = index

    def predict(self, queies):
        predictions = [self.inverted_index_score(i) for i in  queies]
        return predictions

    def inverted_index_score(self, query_sent):
        """
        now we implement inverted index to handle query
        
        :param query_sent: 
        :return: 
        
        """
        query_words = my_tokenizer(query_sent)
        score = defaultdict(float)

        for w in query_words:
            try:
                col_i = self.features[w]
                inverted_ix = self.collection_matrix[:, col_i]
                for di in range(inverted_ix.shape[0]):
                    score[di] += inverted_ix[di, 0]
            except KeyError:
                pass

        index_score = sorted(score.items(), key=lambda (k, v): v, reverse=True)

        if index_score:
            top10_doc_index = [i[0] for i in index_score[:10]]
            return top10_doc_index
        else:
            print 'error occured' ,query_sent
            return -1, 0

In [4]:
def build_model_and_evaluate(model, query ,report=False):
    evaluate_row = []
    pred = model.predict(query)
    quest_index = 0
    for pred_index in pred:
        drow = dict()
        if report:
            print pred_index
        drow['question_ID'] = quest_index
        drow['prediction_ID'] = pred_index
        evaluate_row.append(drow)
        quest_index += 1
    return evaluate_row


# #build model for each document collaction

for document in dataset:
    tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=1, use_idf=True,norm='l1',stop_words=None, tokenizer=my_tokenizer)
    document_collections_sents = document['sentences']
    tfidf_matrix = tfidf_vectorizer.fit_transform(document_collections_sents)
    document['model'] = MostRelevantSentenceModel(vectorizer=tfidf_vectorizer,collection_matrix=tfidf_matrix)

#### Output with predtion and actual values

In [5]:
import csv
csv_file = open('evaluatin_test_results.csv', mode='w',)
fieldnames = ['document_ID', 'question_ID','question','prediction_ID','prediction_sentence']
writer = csv.DictWriter(csv_file, fieldnames=fieldnames, )
writer.writeheader()

ddi = 0
for document in dataset:
    questions = [i['question'] for i in document['qa']]
    model = document['model']
    result_row = build_model_and_evaluate(model, questions)
    doc_sents = document['sentences']
    for r in result_row:
        r['document_ID'] = ddi
        r['question'] = questions[r['question_ID']].encode('utf-8')
        r['prediction_sentence'] = doc_sents[r['prediction_ID'][0]].encode('utf-8')
        writer.writerow(r)
    ddi += 1
print 'Running time count:', time.time() - t0

error occured What was destroyed in the fire?
error occured What is the Barengraben?
error occured Who were the “intelligentia?” 
error occured What are phrynges?
error occured When was the shooting?
Running time count: 206.378999949


In [76]:
def detectQuestion(argument):
    argument = tokenizeUnicode(argument.lower())
    if 'what' in argument and len(set(argument) & set(['year','time','rankings']))!=0:
        return 'NUMBER'
    elif 'who' in argument or 'whom' in argument or 'whose' in argument or \
            ('what' in argument and len(set(argument) & set(['name']))!=0):
        return 'PERSON'
    elif 'when' in argument:
        return 'NUMBER'
    elif 'where' in argument:
        return 'LOCATION'
    elif 'how' in argument and len(set(argument) & set(['many','much','long','far']))!=0:
        return 'NUMBER'        
    elif 'why' in argument:
        return 'O'
    elif 'which' in argument and len(set(argument) & set(['year',"years"]))!=0:
        return 'NUMBER'
    else:
        return 'O'

'PERSON'

In [132]:
def tokenizeUnicode(aUnicode):
    return word_tokenize(re.sub(',', '',re.sub('[^a-zA-Z0-9-_*., ]', ' ',aUnicode)))

from nltk.tag.stanford import StanfordPOSTagger
cwd = os.getcwd()
stPOS = StanfordPOSTagger(cwd+'\\data\\wsj-0-18-left3words-distsim.tagger',cwd+'\data\stanford-postagger.jar')
def posTagger(aString):
    return stPOS.tag_sents([tokenizeUnicode(aString)])[0]

In [52]:
with open("NERtest.json") as json_file:
    json_data = json_load_byteified(json_file)
print 'import success'

import success


In [111]:
csv_file = 'evaluatin_test_results.csv'
i=0
answerSecondFilter = []
dictDoc={}
question={}
with open(csv_file) as csvfile:
    readCSV = csv.DictReader(csvfile, delimiter=',')
    for row in readCSV:        
        document_i = int(row['document_ID'])
        question_i = int(row['question_ID'])
        filteredlistOfNERSentence = []
        question_type = detectQuestion(row['question'])
        question[document_i,question_i] = row['question']
        for x in ast.literal_eval(row['prediction_ID']):
            listOfNERSentence = json_data[document_i][1][x]                      
            if question_type in [k[1] for k in listOfNERSentence]:
                filteredlistOfNERSentence.append(x)
                i+=1
        dictDoc[document_i,question_i,question_type]=filteredlistOfNERSentence        
print len(dictDoc)

8974


[(u'what', u'WP'), (u'is', u'VBZ')]

In [79]:
dictDoc2 = {}
for i,j,qtype in dictDoc:
    if qtype!= u'O':
        for x in dictDoc[i,j,qtype]:
            dictDoc2[i,j]=[x for x in json_data[i][0][j] if x[1]==qtype]
    else:
        for x in dictDoc[i,j,qtype]:
            dictDoc2[i,j]=json_data[i][0][j][0]

In [133]:
print dictDoc[0,0,u'NUMBER'],dictDoc2[0,0]
print json_data[0][0][0]
print json_data[0][1][353]
print json_data[0][1][39]
print posTagger(question[0,0])
print [x for x in posTagger(question[0,0]) if x[1]=='NNP']


[353, 39] []
[['What', 'O'], ['year', 'O'], ['did', 'O'], ['the', 'O'], ['Crimean', 'O'], ['War', 'O'], ['begin', 'O']]
[['The', 'O'], ['Crimean', 'O'], ['War', 'O'], ['marked', 'O'], ['the', 'O'], ['ascendancy', 'O'], ['of', 'O'], ['France', 'LOCATION'], ['to', 'O'], ['the', 'O'], ['position', 'O'], ['of', 'O'], ['pre-eminent', 'O'], ['power', 'O'], ['on', 'O'], ['the', 'O'], ['Continent', 'O'], ['411', 'NUMBER'], ['the', 'O'], ['continued', 'O'], ['decline', 'O'], ['of', 'O'], ['the', 'O'], ['Ottoman', 'O'], ['Empire', 'O'], ['and', 'O'], ['the', 'O'], ['beginning', 'O'], ['of', 'O'], ['a', 'O'], ['decline', 'O'], ['for', 'O'], ['Tsarist', 'O'], ['Russia', 'LOCATION'], ['.', 'O']]
[['In', 'O'], ['1830', 'NUMBER'], ['Greece', 'LOCATION'], ['becomes', 'O'], ['an', 'O'], ['independent', 'O'], ['state', 'O'], ['after', 'O'], ['10', 'NUMBER'], ['years', 'O'], ['of', 'O'], ['independence', 'O'], ['war', 'O'], ['and', 'O'], ['the', 'O'], ['Russo-Turkish', 'O'], ['War', 'O'], ['of', 'O'], ['

### Entity Extraction

In [60]:
def json_load_byteified(file_handle):
    return _byteify(json.load(file_handle, object_hook=_byteify),ignore_dicts=True)

def _byteify(data, ignore_dicts = False):
    # if this is a unicode string, return its string representation
    if isinstance(data, unicode):
        return data.encode('utf-8')
    # if this is a list of values, return list of byteified values
    if isinstance(data, list):
        return [ _byteify(item, ignore_dicts=True) for item in data ]
    # if this is a dictionary, return dictionary of byteified keys and values
    # but only if we haven't already byteified it
    if isinstance(data, dict) and not ignore_dicts:
        return {
            _byteify(key, ignore_dicts=True): _byteify(value, ignore_dicts=True)
            for key, value in data.iteritems()
        }
    # if it's anything else, return it in its original form
    return data

with open("data/QA_train.json") as json_file:
    json_data = json_load_byteified(json_file)
print 'import success'

import success


In [ ]:
import os
java_path = "C:/Program Files/Java" # replace this
os.environ['JAVAHOME'] = java_path

In [ ]:
!pip uninstall nltk
!pip install nltk

In [61]:
cwd = os.getcwd()
st = StanfordNERTagger(cwd+'\data\english.all.3class.distsim.crf.ser.gz',cwd+'\data\stanford-ner.jar')

if not os.path.isfile("NERtrain.json"):    
    start = time.time()
    progressT = len(json_data)    
    listOfDocument=[]
    i=0
    for jd in json_data:
        aList=[]        
        aList.append(st.tag_sents([word_tokenize(re.sub(',', '',re.sub('[^a-zA-Z0-9-_*., ]', ' ',x['question']))) for x in jd['qa']]))
        #remove the below file if running on test set
        aList.extend([st.tag_sents([word_tokenize(re.sub(',', '',re.sub('[^a-zA-Z0-9-_*., ]', ' ',x['answer']))) for x in jd['qa']])])
        aList.append(st.tag_sents([word_tokenize(re.sub(',', '',re.sub('[^a-zA-Z0-9-_*., ]', ' ',x))) for x in jd['sentences']]))
        listOfDocument.append(aList)
        i+=1
        sys.stdout.write('\r')
        sys.stdout.write("%d%%" % (i*100/progressT))
        sys.stdout.flush()    
    for document in range(0,len(listOfDocument)):
        #change [2] to [1] if test set
        for sentence in range(0,len(listOfDocument[document][2])):
            for word in range(0,len(listOfDocument[document][2][sentence])):   
                listOfDocument[document][2][sentence][word]= (listOfDocument[document][2][sentence][word][0]
                                                              ,listOfDocument[document][2][sentence][word][1] if not listOfDocument[document][2][sentence][word][0].isdigit() else u'NUMBER')
    with open('NERtrain.json', 'w') as outfile:
        json.dump(listOfDocument, outfile)
    end = time.time()
    print '\nTime spending:',end - start    
else:    
    print 'there is a file'
with open("NERtrain.json") as json_file:
        json_dataNER = json_load_byteified(json_file)

100%
Time spending: 2928.25099993


In [59]:
listOfDocument[0][0][0]

[(u'What', u'O'),
 (u'year', u'O'),
 (u'did', u'O'),
 (u'the', u'O'),
 (u'Crimean', u'O'),
 (u'War', u'O'),
 (u'begin', u'O')]

In [ ]:
for document in range(0,len(listOfDocument)):
        for sentence in range(0,len(listOfDocument[document])):
            for word in range(0,len(listOfDocument[document][2][sentence])):   
                listOfDocument[document][2][sentence][word]= (listOfDocument[document][2][sentence][word][0],listOfDocument[document][2][sentence][word][1] if not listOfDocument[document][2][sentence][word][0].isdigit() else u'NUMBER')